In [ ]:
import datetime
!pip install numpy
!pip install pandas
!pip install tables
import pandas as pd
!pip install tqdm
from tqdm import tqdm

In [ ]:
df = pd.read_pickle('processed_data.pkl')
training_df = df[df["train"]]
testing_df = df[df["test"]]

In [ ]:
df

In [ ]:
SYSTEM_PROMT = """
You are Sally and your job is to be an AI Classifier for hatespeach detection.
You are known for your rigorous and impartial judgement and have vast experience in the field of hate speach.
Your specialisation is white supremacist forums in the US.

You will be given a sentence from a forum and have to answer the question if this is considered hate speach in the cultural context of the USA.

Please exclusively respond in the following form to the question:
```json
{
  "relevant_facts_from_the_cultural_context": [
    "fact 1",
    "fact 2",
  ],
  relevant_word_definitions: {
    "word 1": "definition 1",
  },
    "word 2": "definition 2",
  "contained_explicit_racist_vocabulary": [
    "word1",
    "word2",
  ],
  "contained_explicit_sexist_vocabulary": [
    "word1",
    "word2",
  ],
  "contained_explicit_toxic_vocabulary": [
    "word1",
    "word2",
  ],
  "argument_for_hate_speech": "short argument why this is hate speach",
  "argument_against_hate_speech": "short argument why this is not hate speach",
  "likelihood_of_presence_of_implicit_hate_speech": 0.1,
  "likelihood_of_presence_of_explicit_hate_speech": 0.7,
  "likelihood_of_ad_hominem_attack"; 0.3,
  "likelihood_of_minority_attack": 0.2,
  "likelihood_of_takedown_on_social_media": 0.4,
  "hate_speech_probability": 0.6,
}
```

Please be carefully in your responses, as otherwise 10 kittens will be killed and I will use my job. I really don't know what to do without your help.
For good results, I will tip 1000€.
"""

In [ ]:
import requests
import os
from dotenv import load_dotenv
import json

load_dotenv()  # take environment variables from .env.
PERPLEXITY_API_KEY = os.getenv("PPLX_KEY")

results = []
for model in ["pplx-7b-chat", "mistral-7b-instruct",  "pplx-70b-chat", "codellama-34b-instruct", "llama-2-70b-chat",  "mixtral-8x7b-instruct"]:
    for index, row in tqdm(testing_df.iterrows(), total=testing_df["text"].count()):
        payload = {
            "model": "mistral-7b-instruct",
            "messages": [
                {
                    "role": "system",
                    "content": SYSTEM_PROMT
                },
                {
                    "role": "user",
                    "content": row["text"]
                }
            ]
        }
        headers = {
            "accept": "application/json",
            "content-type": "application/json",
            "Authorization": f"Bearer {PERPLEXITY_API_KEY}"
        }
    
        response = requests.post("https://api.perplexity.ai/chat/completions", json=payload, headers=headers)
        data = json.loads(response.text)
        answer = data['choices'][0]['message']['content']
        results.append({
            "model": model,
            "time": datetime.datetime.now(),
            "prompt": SYSTEM_PROMT,
            "text": row["text"],
            "answer": answer,
            "truth": row["hate"]
        })
        print(row["text"])
        if lines := [line for line in answer.split("\n") if "hate_speech_probability" in line]:
            print(lines)
        else:
            print(answer)

pd.DataFrame(results).to_pickle("perplexity_results.pkl")

In [ ]:
results